<a href="https://colab.research.google.com/github/acastellanos-ie/NLP-MBDS-EN/blob/main/08_agentic_ai/agentic_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧠 Lab Session: Agentic AI with Python Logic & Reasoning

**Objective:**
In this final session, we build an **Autonomous Agent**. Unlike a chatbot that just talks, an Agent can *do things*. We will give our Llama-2 model access to a **Python Interpreter**. This allows the AI to write code to solve logic puzzles, perform complex math, or process data, effectively giving it the capabilities of a Data Scientist.

**The Architecture:**
1.  **Brain:** TinyLlama (Quantized) decides *what* to do.
2.  **Tool A (Memory):** A Vector Database (RAG) for specific facts about our fictional planet.
3.  **Tool B (Logic):** A Python REPL (Read-Eval-Print Loop) to execute code.
4.  **Interface:** Gradio to visualize the Agent's "Thought Process".

## 1. Environment Setup

We need `langchain-experimental` to access the Python execution tools safely.

In [1]:
# @title 🛠️ Install Libraries
!pip install -Uqqq "numpy>=2.0.0" langchain langchain-community langchain-experimental transformers torch accelerate bitsandbytes gradio sentence-transformers faiss-cpu langchain-huggingface
print("Agent Environment set up successfully! Please restart the session if you still see numpy errors.")

Agent Environment set up successfully! Please restart the session if you still see numpy errors.


## 2. The Brain

We load the model in 4-bit mode to fit comfortably in the free Colab GPU. We set `temperature=0.01` because we need the model to be precise when calling tools, not creative.

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from langchain_huggingface import HuggingFacePipeline

# 1. 4-Bit Quantization Config (Efficiency First)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

print(f"Loading {model_id} as the Agent's Brain...")

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

# 2. Text Generation Pipeline
text_generation_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512, # Agents need space to "think"
    temperature=0.01,   # Precision over creativity
    repetition_penalty=1.1,
    return_full_text=False
)

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)
print("Agent Brain Ready!")

Loading TinyLlama/TinyLlama-1.1B-Chat-v1.0 as the Agent's Brain...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cuda:0


Agent Brain Ready!


## 3. Creating the Tools (Hands & Calculator)

We will give the Agent two superpowers:
1.  **Planetary Knowledge (RAG):** To retrieve facts it wasn't trained on.
2.  **Python Interpreter:** To execute code for math and logic.

In [3]:
from langchain_core.documents import Document
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.tools import Tool
from langchain_experimental.tools import PythonREPLTool

# --- TOOL 1: RAG (Planetary Knowledge) ---
planet_data = """
Proxima Centauri b is a habitable planet orbiting the red dwarf Proxima Centauri.
The distance from Earth to Proxima Centauri b is approximately 4.24 light-years.
The average surface temperature is -39 degrees Celsius.
The orbital period (one year) is exactly 11.2 Earth days.
The colonization mission "Exodus-1" arrived in 2150.
The population is currently 4,500 colonists.
The currency is "Stardust-Credits".
"""

# Vector Store Setup
text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=0)
docs = text_splitter.create_documents([planet_data])
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_db = FAISS.from_documents(docs, embeddings)
retriever = vector_db.as_retriever()

knowledge_tool = Tool(
    name="Planetary Database",
    func=retriever.invoke,
    description="Useful for answering factual questions about Proxima Centauri b, its distance, population, or history."
)

# --- TOOL 2: PYTHON REPL (Logic & Math) ---
python_tool = PythonREPLTool()
python_tool.name = "Python Interpreter"
python_tool.description = "A Python shell. Use this to execute python commands. Input should be a valid python command. Useful for complex logic, math, sorting lists, or processing text strings."

# Combine tools
tools = [knowledge_tool, python_tool]

print("Tools created: [Planetary Database, Python Interpreter]")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Tools created: [Planetary Database, Python Interpreter]


## 4. Initializing the Agent (The Orchestrator)

We use the **ReAct (Reason + Act)** framework. The agent will loop through:
* **Thought:** "What should I do?"
* **Action:** "Run this Python code."
* **Observation:** "Here is the result of the code."
* **Final Answer:** "Here is the summary."

In [4]:
from langchain.agents import initialize_agent, AgentType

tools = [knowledge_tool, python_tool]

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True
)

print("Agent Initialized. Ready to Reason and Act.")

ImportError: cannot import name 'initialize_agent' from 'langchain.agents' (/usr/local/lib/python3.12/dist-packages/langchain/agents/__init__.py)

In [ ]:
from langchain.agents.react.agent import create_react_agent
from langchain.agents.agent import AgentExecutor
from langchain import hub

# 1. Obtener el prompt del hub
prompt = hub.pull("hwchase17/react")

# 2. Construir el agente ReAct
agent_runnable = create_react_agent(llm, tools, prompt)

# 3. Crear el AgentExecutor
agent = AgentExecutor(
    agent=agent_runnable,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

print("Agent Initialized using create_react_agent. Ready to Reason and Act.")

## 5. Testing the Logic

Let's verify that the Python tool is working by asking something Llama-2 cannot do purely by text prediction (like iterating through a sequence).

In [ ]:
# Test: Fibonacci Sequence
# The LLM should write a Python loop to solve this.
query = "Generate the first 10 numbers of the Fibonacci sequence and calculate their sum."

print(f"User Query: {query}")
print("-" * 40)
result = agent.run(query)
print("-" * 40)
print(f"Final Answer: {result}")

## 6. The Interface (Gradio)

We will build a Chat Interface that reveals the **"Brain's Monologue"**. We capture the system logs (stdout) to show the user exactly how the Agent decided to write Python code or check the database.

In [ ]:
import gradio as gr
import sys
from io import StringIO

def solve_with_reasoning(message, history):
    # 1. Capture the "Thinking Process" (Stdout)
    old_stdout = sys.stdout
    sys.stdout = mystdout = StringIO()

    try:
        # 2. Run the Agent
        result = agent.run(message)
    except Exception as e:
        result = f"I encountered an error: {str(e)}"

    # 3. Restore Stdout & Get Logs
    sys.stdout = old_stdout
    reasoning_logs = mystdout.getvalue()

    # 4. Format Output for the UI
    # We use Markdown to make the code/logs look like a terminal
    formatted_output = (
        f"🧠 **Agent Thought Process:**\n"
        f"```bash\n{reasoning_logs}\n```\n\n"
        f"✅ **Final Answer:**\n{result}"
    )
    return formatted_output

# Create the Gradio Interface
demo = gr.ChatInterface(
    fn=solve_with_reasoning,
    title="🤖 Agentic AI: The Python Coder",
    description="Ask me complex questions. I can use a Database or write Python code to answer.",
    examples=[
        "What is 30% of the population of Proxima Centauri b?",
        "Create a list of numbers from 1 to 10, square them, and find the average.",
        "How many seconds does it take for light to travel from Earth to Proxima b? (Use Python)",
        "Sort this list alphabetically: [Zebra, Apple, Mango, Delta]"
    ],
    theme="glass"
)

demo.launch(share=True, debug=True)